In [ ]:
knitr::opts_chunk$set(echo = TRUE)



## Introduction

In this EDA, I explore a dataset of airline on-time performance to try and find insights to flight delay and cancellations. The dataset used is a very large dataset that consists of flight arrival and departure details for all commercial flights within the USA, from October 1987 to April 2008. There are over 120 million observations (flights) in this dataset for flights. The data was compressed into individual CSV files for each year.

I chose to explore this particular dataset because it would allow me learn new skills and optimization techniques for handling large datasets.

Due to the size of this dataset, it would very difficult to load the data into a Pandas dataframe in memory without reducing it to a very small subset of the data, so I decided to employ the use of R markdown (instead of Jupyter notebook) so that I can use R packages, along with SQL queries, to wrangle the data into a more summarized format that a Pandas dataframe can handle.

## Preliminary Wrangling

### Importing R Libraries


In [ ]:
library(tidyverse)
library(skimr)
library(dplyr)
library(here) # To locate files based on current working directory
library(janitor) # Tools for for examining and cleaning dirty data.
library(reticulate) # For reading R objects in Python
library(data.table) # For reading large datasets efficiently
library(inborutils) # For reading CSV files and converting to SQL
library(DBI) # Interface to connect with SQL databases
library(RSQLite) # For connecting with SQL databases


### Importing Python Packages



In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pathlib
import os


### Loading In The Data

This dataset contains 21 large CSV files of flight data for each year from 1987 to 2008, as well as some other CSV files for which contain extra information. I will not directly read any of the large CSV files because it would take too much memory. I will read in the other CSV files, then use R libraries and SQL queries to read in smaller samples of the data to explore.


In [ ]:
path_column_data = "airline/airline_dataset_column_info.csv"
path_airport_data = "airline/other_data/airports.csv"
path_carrier_data = "airline/other_data/carriers.csv"
path_plane_data = "airline/other_data/plane-data.csv"
path_main = "airline/main_data" # Path to the main (yearly) CSV files

def absolute_file_paths(directory):
    data = {}
    files = os.listdir(directory)
    paths = [f"{path_main}/{file}" for file in files]
    
    for idx in range(len(files)):
        name = files[idx].split('.')[0]
        data[name] = paths[idx]
    return data

main_files = absolute_file_paths(path_main)


In [ ]:
# Data on column descriptions for the main files
column_data = pd.read_csv(path_column_data)

# Data on different airports
airport_data = pd.read_csv(path_airport_data)

# Airline companies
carrier_data = pd.read_csv(path_carrier_data)

# Plane data, specifications and other info
plane_data = pd.read_csv(path_plane_data)


In [ ]:
# Info on column descriptions for the main files
column_data <- read.csv(py$path_column_data)

# Data on different airports
airport_data <- read.csv(py$path_airport_data)

# Information on airline companies
carrier_data <- read.csv(py$path_carrier_data)

# Plane data, specifications and other info
plane_data <- read.csv(py$path_plane_data)


For the main data, I have written a script that reads in the data in smaller chunks and stores them in a database file (sqlite). Each year's data is stored in its own table. I also store the other data in their own tables so that later, when needed, I can reference them using SQL joins. The process takes a while to run because of the large dataset (over 30 minutes on my PC).

Also, the original files are named by the year they represent. It is not be good practice to name a database table starting with a number, so the script adds a prefix to each name.

For now, I have added a condition so the code will only be executed if the sqlite file is not detected in the project root directory.


In [ ]:
path_main <- "airline/main_data"
db_file <- "airline_data.sqlite"

save_in_sql <- function() {
  main_files <- list.files(path = path_main, full.names = TRUE)
  
  if (!file.exists(db_file)) {
    # Creating the airport data table
    inborutils::csv_to_sqlite(
                csv_file = py$path_airport_data,
                table_name = "airports",
                sqlite_file = db_file,
                show_progress_bar = FALSE)
    
    # Creating the carrier data table
    inborutils::csv_to_sqlite(
                csv_file = py$path_carrier_data,
                table_name = "carriers",
                sqlite_file = db_file,
                show_progress_bar = FALSE)
    
    # Creating the plane data table
    inborutils::csv_to_sqlite(
                csv_file = py$path_plane_data,
                table_name = "planes",
                sqlite_file = db_file,
                show_progress_bar = FALSE)
    
    # Creating the tables for each of the years' data
    for (csv in main_files) {
      csv_name <- strsplit(csv, "/|[.]")[[1]] # Splitting the csv name by "/" or "."
      csv_name <- csv_name[length(csv_name)-1] # Getting the second last element of the list
      table_name <- paste("table", csv_name, sep="_")
    
      print("Updating table: %s", table_name)
      inborutils::csv_to_sqlite(
                  csv_file = csv,
                  sqlite_file = db_file, 
                  table_name = table_name, 
                  pre_process_size = 1000,
                  chunk_size = 50000, 
                  show_progress_bar = TRUE)
    }
    
  }
}

save_in_sql()


Now I inspect the database file to be sure that all tables have been added and updated properly



In [ ]:
airline_db <- dbConnect(SQLite(), db_file) # Making a connection to db

db_tables <- dbListTables(airline_db) # List out the tables in the db
print(db_tables)

db_1993_cols <- dbListFields(airline_db, "table_1993") # Column names for specific table in db
print(length(db_1993_cols))


#### Structure of the dataset

We can see from the above result that there are 29 columns in the table and this is the same across all the tables (the yearly tables), they all have the same columns, but we don't know exactly how many rows are in each table.

The code below is a script/query to return exactly the number of rows (observations) that are in each table. The query can take a few minutes to execute the first time.


In [ ]:
# count_rows <- function() {
#     Table = character() # Empty vector/list to store table names
#     Row_Count = integer() # Empty vector/list to store row counts
#   
#     for (table in db_tables) {
#         query_rows <- sprintf("SELECT COUNT(*) AS Rows FROM %s", table)
#         row_count <- dbGetQuery(airline_db, query_rows)[[1]]
#     
#         Table <- c(Table, table) # Appending each table name to the vector
#         Row_Count <- c(Row_Count, row_count) # Appending each row count to the vector
#     }
#   
#     df_row_count <- data.frame(Table, Row_Count)
#     return(df_row_count)
# }
# table_row_count <- count_rows()
# table_row_count


We can now see the number of rows in each table, which sums up to over 120 million observations. To test the SQL connection, I load in the first 500 rows of data from a particular year (2005 dataset in this case) using SQL and the R interface



In [ ]:
query_test <- "SELECT * FROM table_2005 LIMIT 500"

tbl(airline_db, sql(query_test)) # Runs the query and displays results without loading it in memory

top_rows <- dbGetQuery(airline_db, query_test) # Runs the query and stores it in a dataframe, in memory♂

dbDisconnect(airline_db) # Disconnect from the database when done


Now lets look at some summary statistics for the data. I will read in the first 5,000 rows of the 2007 flight data.



In [ ]:
top_2007 = pd.read_csv(main_files['2007'], nrows=5000)
top_2007.info()


Most of the columns are numeric, some indicating arrival and departure, as well as different causes of delays. There are some binary columns such as "Cancelled" and "Diverted" which are important variables to analyze.

#### Features of interest

For this EDA, I am interested in exploring some of the ideas suggested on the source website which are:

1.  When is the best time of day/day of week/time of year to fly to minimize delays?
2.  Do older planes suffer more delays?
3.  How well does weather predict plane delays?

Generally, I am interested in exploring the cause of flight delays and cancellations.

#### Areas To Focus On

For this investigation, I will get the best insights by focusing on the "delay" columns. By analyzing the delays on each day of the week and each month, I believe I can get a good idea of the best times to fly. I will explore data for a single year. Then later on, I will compare the data across the other years to see if there are similar patterns across the years.

## Univariate Exploration

I will start by analyzing the departure delays: `DepDelay`. I will be using the 2007 dataset. Because the data is so large, I will only read in some select columns into the dataframe.


In [ ]:
df_2007 = pd.read_csv(main_files['2007'], usecols = ['Month', 'DayofMonth', 'ArrDelay', 'DepDelay'])
df_2007.info()
df_2007.describe()

# Check for missing values
df_2007['DepDelay'].isnull().sum()
df_2007[df_2007.ArrDelay.notnull()]


In [ ]:
# Changing column data types to reduce memory usage
df_2007 = df_2007.astype({'Month':'int8', 'DayofMonth':'int8', 'ArrDelay':'float32', 'DepDelay':'float32'})


In [ ]:
binsize = 30
bins = np.arange(0, df_2007['DepDelay'].max()+binsize, binsize)

plt.figure(figsize=[14, 8])
plt.hist(data = df_2007, x = 'DepDelay', bins = bins)
plt.xlabel('Departure Delays (mins)')
plt.show()


The distribution is skewed to the left and there is a short tail. A large majority of the data falls within the range of 0 and 250 minutes. I would have gone for a logarithmic scale but this data has negative values (because there are flights that took off before the expected departure time). There are also some missing values.



In [ ]:
# Flights that took off over 25 minutes earlier
df_2007[df_2007.DepDelay < -25].info()

# Lowest departure delay
df_2007[df_2007.DepDelay < 0].DepDelay.min()


We can see that there are many flights that took off before the expected departure time (almost 50% of all the flights in that year). That is not unusual, especially if it falls within a few minutes and all passengers are available, but there are many flights that took off unusually early (over 30 minutes early, even up to 5 hours early). There can be many reasons for this but for now since this exploration is mainly focused on delay times and there are so many records to work with, I will only assess flights that were actually delayed,



In [ ]:
df_2007_delayed = df_2007[df_2007.DepDelay > 0]
df_2007_delayed.info()
df_2007_delayed.describe()


Now I will try to plot using a log scale



In [ ]:
log_binsize = 0.025
bins = 10 ** np.arange(0, np.log10(df_2007_delayed['ArrDelay'].max())+log_binsize, log_binsize)

plt.figure(figsize=[14, 8])
plt.hist(data = df_2007_delayed, x = 'ArrDelay', bins = bins)
plt.xscale('log')
# plt.xticks([500, 1e3, 2e3, 5e3, 1e4, 2e4], [500, '1k', '2k', '5k', '10k', '20k'])
plt.xlabel('Arrival Delays (mins)')
plt.show()


## Bivariate Exploration

I will look at the relationship between departure delays and arrival delays. Ideally I would expect this to be positive linear correlation very close to an 'r' value of 1 since a flight that leaves late is expected to arrive late. I know there are exceptions to this, like when there is a fault midair, unexpected weather, or when a flight gets diverted, but I expect these to be relatively few and not affect the positive correlation significantly.


In [ ]:
# Dropping rows with missing values
print("Original rows and columns =",df_2007.shape)
df_2007_sampled = df_2007.dropna(subset=['DepDelay', 'ArrDelay']).sample(n=1000, replace = False)
print("Sampled rows and columns =",df_2007_sampled.shape)

print(df_2007_sampled.info())
print(df_2007_sampled.head())

plt.figure(figsize=[12, 12])
plt.scatter(data=df_2007_sampled, x='DepDelay', y='ArrDelay')
plt.show()


Since there are a lot of overlapping points, I will apply some transparency to get a better picture.



In [ ]:
plt.figure(figsize=[12, 12])
sns.regplot(data=df_2007_sampled, x='DepDelay', y='ArrDelay', scatter_kws = {'alpha': 1/5}, fit_reg = False);
plt.show()


Most of the data falls within the range of 200 so I will zoom in on that group.



In [ ]:
df_2007_sampled = df_2007_sampled.query("DepDelay < 200")

plt.figure(figsize=[12, 12])
sns.regplot(data=df_2007_sampled, x='DepDelay', y='ArrDelay', scatter_kws = {'alpha': 1/5}, fit_reg = False);
plt.show()


This is in line with my predictions so I will go ahead and analyze other variables, since I am sure that the delay times are consistent.

#### Day of Week Analysis

Next, I will analyze the delays on a `day-of-week` basis.


In [ ]:
get_mean_delay_year <- function(year) {
    airline_db <- dbConnect(SQLite(), db_file) # Making a connection to db
    
    # Query for mean delay times in January (actual delays, no early flights)
    query_jan <- sprintf("SELECT DayOfWeek, DayOfMonth, 
                                 AVG(DepDelay) AS MeanDepDelay, 
                                 AVG(ArrDelay) AS MeanArrDelay
                          FROM table_%s
                          WHERE DepDelay > 0 AND ArrDelay > 0
                          GROUP BY DayOfWeek, DayOfMonth", year)
    
    # tbl(airline_db, sql(query_jan)) # Runs the query and displays results without loading it in memory
    delays_2007 <- dbGetQuery(airline_db, query_jan) # Runs the query and stores it in a dataframe, in memory♂
    dbDisconnect(airline_db) # Disconnect from the database when done
    return(delays_2007)
}


In [ ]:
delays_2007 <- get_mean_delay_year('2007')
str(delays_2007)
skim(delays_2007)


For the `DayOfWeek` data, I will make another column in the dataframe that shows the text representation (Monday, Tuesday ...) so that it would be easier to understand in the plot.



In [ ]:
def change_column_type(df):
    # Converting R dataframe to Pandas dataframe
    df_delays = pd.DataFrame(df)
    
    # Changing day and month columns from float to integer data types
    df_delays = df_delays.astype({'DayOfWeek':'int8', 'DayofMonth':'int8'})
    
    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    
    # Creating the new column
    df_delays['Day'] = df_delays['DayOfWeek'].apply(lambda x: days_of_week[x-1])
    
    return(df_delays)

delays_2007 = change_column_type(r.delays_2007)
delays_2007.head()


In [ ]:
plt.figure(figsize=[16,12])
sns.barplot(data=delays_2007, x='Day', y='MeanDepDelay')
plt.title("Average Flight Delay Times In 2007")
plt.xlabel("Day of Week")
plt.ylabel("Average Departure Delay (min)")
plt.show();


From the above chart alone, we can see that there is a wider range of delay (in minutes) on Sundays, followed by Wednesdays and Tuesdays. Also, there is at least one flight on a Sunday that was delayed for over 55 minutes. The days with the least median (50th percentile) delay times are Tuesday and Saturday (around 35 minutes). I will see if there is a consistent theme across the years by plotting the chart for 9 consecutive years (1999 - 2008)

**I left the colors because, even though the weekday variable is ordinal (i.e. Tuesday comes after Monday and so on), the order doesn't really matter much in this case because, for example, Friday is not better than Sunday, Monday is not higher than Saturday, etc. The colors will be helpful in identifying each weekday in the subsequent plots.**


In [ ]:
delays_2000 <- get_mean_delay_year('2000')
delays_2001 <- get_mean_delay_year('2001')
delays_2002 <- get_mean_delay_year('2002')
delays_2003 <- get_mean_delay_year('2003')
delays_2004 <- get_mean_delay_year('2004')
delays_2005 <- get_mean_delay_year('2005')
delays_2006 <- get_mean_delay_year('2006')
delays_2007 <- get_mean_delay_year('2007')
delays_2008 <- get_mean_delay_year('2008')


In [ ]:
delays_2000 = change_column_type(r.delays_2000)
delays_2001 = change_column_type(r.delays_2001)
delays_2002 = change_column_type(r.delays_2002)
delays_2003 = change_column_type(r.delays_2003)
delays_2004 = change_column_type(r.delays_2004)
delays_2005 = change_column_type(r.delays_2005)
delays_2006 = change_column_type(r.delays_2006)
delays_2007 = change_column_type(r.delays_2007)
delays_2008 = change_column_type(r.delays_2008)


In [ ]:
fig, ax = plt.subplots(ncols = 3, nrows = 3 , figsize = [17,17])

sns.barplot(data=delays_2000, x='Day', y='MeanDepDelay', ax = ax[0, 0])
# plt.title(f"Average Flight Delay Times In {delay_tables}")
sns.barplot(data=delays_2001, x='Day', y='MeanDepDelay', ax = ax[1, 0])
# plt.title(f"Average Flight Delay Times In {delay_tables}")
sns.barplot(data=delays_2002, x='Day', y='MeanDepDelay', ax = ax[2, 0])
# plt.title(f"Average Flight Delay Times In {delay_tables}")
sns.barplot(data=delays_2003, x='Day', y='MeanDepDelay', ax = ax[0, 1])
# plt.title(f"Average Flight Delay Times In {delay_tables}")
sns.barplot(data=delays_2004, x='Day', y='MeanDepDelay', ax = ax[1, 1])
# plt.title(f"Average Flight Delay Times In {delay_tables}")
sns.barplot(data=delays_2005, x='Day', y='MeanDepDelay', ax = ax[2, 1])
# plt.title(f"Average Flight Delay Times In {delay_tables}")
sns.barplot(data=delays_2006, x='Day', y='MeanDepDelay', ax = ax[0, 2])
# plt.title(f"Average Flight Delay Times In {delay_tables}")
sns.barplot(data=delays_2007, x='Day', y='MeanDepDelay', ax = ax[1, 2])
# plt.title(f"Average Flight Delay Times In {delay_tables}")
sns.barplot(data=delays_2008, x='Day', y='MeanDepDelay', ax = ax[2, 2])
# plt.title(f"Average Flight Delay Times In {delay_tables}")

plt.show();


From a high level overview of the chart above, there is no consistent trend to predict which weekdays have more delays. This is expected because there are many other factors to consider like the month, the season, holidays, airport carrier, plane age and global events.

#### Cancelled and Divertd

Now I will examine the cancelled and diverted flights, specifically the unique carriers. I want to see if flights from a carrier tend to get get cancelled or diverted more than others. 

Below is an SQL query to get all flights that were either diverted or cancelled and group them by the flight carrier. On the original table, there is a column for diverted (1 or 0) and another column for cancelled (1 or 0). I believe the data is not completely tidy because a flight that is cancelled cannot be diverted and vice-versa. So I combined them to a single column indicating whether the flight was cancelled or diverted.


In [ ]:
airline_db <- dbConnect(SQLite(), db_file) # Making a connection to db

get_changed_flights <- function(year) {
    
    query <- sprintf("SELECT 
                        UniqueCarrier,
                        Description AS Carrier,
                        CASE 
                            WHEN Diverted = 1 AND Cancelled = 0 THEN 'Diverted'
                            WHEN Diverted = 0 AND Cancelled = 1 THEN 'Cancelled'
                        END AS FlightChange,
                        COUNT(*) AS Flights
                      FROM table_%s
                      LEFT JOIN carriers
                      ON table_%s.UniqueCarrier = carriers.Code
                      WHERE Diverted = 1 OR Cancelled = 1
                      GROUP BY UniqueCarrier, FlightChange
                      ORDER BY UniqueCarrier", year)
    
    flights_changed <- dbGetQuery(airline_db, query) # Runs the query and stores it in a dataframe, in memory♂
    dbDisconnect(airline_db) # Disconnect from the database when done
    return(flights_changed)
}

flights_changed_2007 = get_changed_flights('2007')
flights_changed_2007



There are 138,120 records of cancelled or diverted flights in 2006. 


In [ ]:
flights_changed_2007 = pd.DataFrame(r.flights_changed_2007)

flights_changed_2007.head()
print(f"Total number of diverted or cancelled flights in 2007: {flights_changed_2007.Flights.sum()}")


In [ ]:
plt.figure(figsize=[16,12])
sns.barplot(data = flights_changed_2007, x = 'UniqueCarrier', y = 'Flights', hue = 'FlightChange')
plt.title("Cancelled & Diverted Flights From Each Carrier In 2007")
plt.show()


We can already see that `MQ` had the highest number of cancelled flights (by a relatively wide margin). To better understand the plot, I will make it horizontal and order it by number of cancelled flights.



In [ ]:
flights_changed_2007 = flights_changed_2007.sort_values(by='Flights', ascending=False)
plt.figure(figsize=[16,12])
sns.barplot(data = flights_changed_2007, y = 'Carrier', x = 'Flights', hue = 'FlightChange')
plt.title("Cancelled & Diverted Flights From Each Carrier In 2007")
plt.show()


I believe its easier to see that the airline companies with the most number of `cancelled` flight is **E** and `diverted` flights are **A B C**. So we can at least infer that the carriers at the bottom of the chart have a good record of flight data. 

Now I'm going to plot the chart for the most recent 6 years from the dataset, to see if this trend is the same across the years, `i.e, to see if the same companies are always on top.`


The above is a plot of average delay times for each day of the week of all the flights in 2007 (the 2007 table). Just judging by this plot alone, we can see that the longest delays occurred on Wednesdays, followed by Thursdays and Fridays. The days with the shortest delays are Mondays and Tuesdays. We can also see a the highest bulge in on Saturdays indicating the highest frequency of delays

